In [1]:
import sys
import numpy as np
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import ArmActionMode, JointVelocity, JointPosition, EndEffectorPoseViaPlanning, EndEffectorPoseViaIK
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.observation_config import ObservationConfig, CameraConfig
from rlbench.tasks import ReachTarget, PickAndLift, StackBlocks, PushButton, StackBlocks, PickUpCup, PlaceHangerOnRack
import matplotlib.pyplot as plt
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from PIL import Image
import torch
from transformers import BitsAndBytesConfig
from pyquaternion import Quaternion
from rlbench.backend.robot import Robot
from scipy.spatial.transform import Rotation
from rlbench.backend.scene import Scene
from pathlib import Path
import os, json

from transformers import AutoModelForVision2Seq
from peft import PeftModel
import argparse
import torch
from vla.action_tokenizer import RLbenchActionTokenizer
from vla.dataset import RLbenchDataset

/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model_path = "/media/lawrence/Work/checkpoints/openvla-7b"
adapter_path = "adapter-tmp/openvla-7b+pick_up_cup+e3+b8+lr-2e-05+lora-r8+dropout-0.0+q-4bit"

In [3]:
processor = AutoProcessor.from_pretrained(base_model_path, trust_remote_code=True)
action_tokenizer = RLbenchActionTokenizer(processor.tokenizer)

In [4]:
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4", #llm_int8_skip_modules = ['projector'],
        )
base_model = AutoModelForVision2Seq.from_pretrained(
        base_model_path,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        quantization_config=quantization_config,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        device_map = "auto"
    )
vla = PeftModel.from_pretrained(base_model, adapter_path, offload_buffers=True)
vla = vla.merge_and_unload()


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:35<00:00, 11.84s/it]
/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [5]:
from vla.base_prompter import PurePromptBuilder
vla_dataset = RLbenchDataset(
    "./datasets/pick_up_cup/data.pt",
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
    prompt_builder_fn=PurePromptBuilder,
)

In [6]:
test_action = torch.tensor([1,-0.5,0.5,-3.14,0,3.14,1])
processor.tokenizer(action_tokenizer(test_action))

{'input_ids': [1, 29871, 31697, 31698, 31798, 31898, 31948, 31997, 31999], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
prompt = "In: What action should the robot take to <INSTRUCTION>?\nOut:"
# prompt = prompt.replace("<INSTRUCTION>", instr.lower())
image = Image.fromarray(np.random.random([224,224]))
inputs = processor(prompt, image).to(vla.device, dtype=torch.bfloat16)
inputs['input_ids'] = torch.cat((inputs['input_ids'], torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(inputs['input_ids'].device)), dim=1)

In [8]:
action_dict = vla.generate(**inputs, max_new_tokens = 7, output_scores = True, return_dict_in_generate=True, do_sample=True)

In [9]:
processor.tokenizer.decode(torch.tensor([31879, 31857, 31889, 31885, 31877, 31870, 31744]))

'▓ਿദ巴飛Ħ忠'

In [20]:
class Agent(object):
    def __init__(self, vla, processor, action_tokenizer):
        self.vla = vla
        self.processor = processor
        self.action_tokenizer = action_tokenizer
        
    def act(self, obs, instr):
        prompt = "In: What action should the robot take to <INSTRUCTION>?\nOut:"
        prompt = prompt.replace("<INSTRUCTION>", instr.lower())
        image = Image.fromarray(obs.front_rgb)
        inputs = self.processor(prompt, image).to(self.vla.device, dtype=torch.bfloat16)
        inputs['input_ids'] = torch.cat((inputs['input_ids'], torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(inputs['input_ids'].device)), dim=1)        
        action_dict = self.vla.generate(**inputs, max_new_tokens = 7, output_scores = True, return_dict_in_generate=True)
        ation_score = torch.stack(action_dict['scores']).squeeze(1)[:,self.action_tokenizer.action_token_begin_idx:self.processor.tokenizer.vocab_size]
        pred_action = self.action_tokenizer.decode_token_score_to_actions(ation_score, soft = True)
        return pred_action



In [21]:
agent = Agent(vla, processor, action_tokenizer)

In [12]:
camera = CameraConfig(image_size=(224, 224), depth=False, point_cloud=False, mask=False)
obs_config = ObservationConfig(left_shoulder_camera=camera, right_shoulder_camera=camera, front_camera=camera, overhead_camera=camera)

env = Environment(
    action_mode=MoveArmThenGripper(
        arm_action_mode=EndEffectorPoseViaPlanning(absolute_mode=True, collision_checking=False), gripper_action_mode=Discrete()),
    obs_config=obs_config,
    headless=False)
env.launch()



In [13]:
task = env.get_task(PickUpCup)

In [14]:
descriptions, obs = task.reset()

In [15]:
action_dict['sequences']

tensor([[    1,   512, 29901,  1724,  3158,   881,   278, 19964,  2125,   304,
           529,  1177, 10810, 29965,  9838, 29958, 29973,    13,  3744, 29901,
         29871, 31912, 31858, 31847, 31899, 31833, 31859, 31761]],
       device='cuda:0')

In [16]:
instr = descriptions[1]
prompt = "In: What action should the robot take to {<INSTRUCTION>}?\nOut:"
prompt = prompt.replace("<INSTRUCTION>", instr.lower())
image = Image.fromarray(obs.front_rgb)

inputs = processor(prompt, image).to(vla.device, dtype=torch.bfloat16)
inputs['input_ids'] = torch.cat((inputs['input_ids'], torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(inputs['input_ids'].device)), dim=1)        
action_dict = vla.generate(**inputs, max_new_tokens = 7, output_scores = True, return_dict_in_generate=True)

agent.act(obs, instr)


tensor([ 0.1250, -0.0350,  1.4250, -3.1102, -2.7332, -0.2827,  1.0000],
       device='cuda:0')

In [17]:
agent.act(obs,descriptions[0])

tensor([ 0.0350,  0.1050,  1.4750,  2.2305, -0.8482, -2.4819,  1.0000],
       device='cuda:0')

In [22]:
task = env.get_task(PickUpCup)
training_steps = 1000
episode_length = 100
obs = None
for i in range(training_steps):
    if i % episode_length == 0:
        print('Reset Episode')
        descriptions, obs = task.reset()
        print(descriptions[1])
    try:
        action = agent.act(obs,descriptions[0]).cpu().numpy()
        action_rotation = Rotation.from_euler('xyz', action[3:6])
        action_quaternion = action_rotation.as_quat()
        # print(delta_quaternion)  # returns (qx, qy, qz, qw)
        action = np.concatenate([action[0:3], action_quaternion, action[-1:]])
        print(action)
        obs, reward, terminate = task.step(action)
        print(reward)
    except Exception as e:
        print(e)
        continue

Reset Episode
grasp the red cup and lift it


/home/lawrence/VLA-RL/VLA-RL/vla/action_tokenizer.py:126: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x_pred = F.softmax(x_score) @ torch.tensor(self.x_bin_centers, dtype=torch.float32).to(device)
/home/lawrence/VLA-RL/VLA-RL/vla/action_tokenizer.py:127: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = F.softmax(y_score) @ torch.tensor(self.y_bin_centers, dtype=torch.float32).to(device)
/home/lawrence/VLA-RL/VLA-RL/vla/action_tokenizer.py:128: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z_pred = F.softmax(z_score) @ torch.tensor(self.z_bin_centers, dtype=torch.float32).to(device)
/home/lawrence/VLA-RL/VLA-RL/vla/action_tokenizer.py:130: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an arg

[ 0.27572709  0.24583329  1.46532702 -0.20408997 -0.25716859 -0.85771209
  0.39565334  0.99771273]
0.0
[ 0.20059341  0.05374869  1.33494139  0.23751288 -0.90787488 -0.24849725
  0.23999988  0.715424  ]
0.0
[ 0.20427898  0.33727607  1.2896384  -0.72007487 -0.00627378 -0.58384206
  0.37494169  0.99982733]
0.0
[ 0.22827733  0.29828966  1.36124277 -0.13839479 -0.44249046 -0.10972493
  0.8792096   0.99659038]
0.0
[ 0.22979419  0.16954969  1.41112947 -0.23194927  0.4435576  -0.86251087
  0.074372    0.92944044]
0.0
[ 0.18916911  0.2723802   1.37525153 -0.47196301 -0.3703273  -0.67230804
  0.43371709  0.88991213]
0.0
[ 0.21629931  0.12004524  1.46467423  0.70475247  0.54476549  0.44603796
 -0.08720467  0.99576843]
0.0
[ 0.25534695  0.23317787  1.34824216  0.01521532  0.74741209 -0.10307417
  0.65613975  0.99922061]
0.0
[ 0.19314648  0.23666552  1.42500782  0.28048186 -0.64344575  0.57425778
  0.4213496   0.96138906]
0.0
[ 0.18919125  0.29094267  1.24881077 -0.86361804 -0.13638627 -0.39551703


KeyboardInterrupt: 

In [ ]:
env.shutdown()

[CoppeliaSim:loadinfo]   done.


In [ ]:
action

array([-0.02001152,  0.02458013, -0.01649398,  0.00402673, -0.01346854,
        0.02819962,  0.99607843])